In [18]:
# https://github.com/pytorch/examples/blob/master/mnist/main.py
from __future__ import print_function
import torch
from torch import nn, optim, cuda
from torch.utils import data
from torchvision import datasets, transforms
import torch.nn.functional as F
import time

# Training settings
batch_size = 64
device = 'cuda' if cuda.is_available() else 'cpu'
print(f'Training MNIST Model on {device}\n{"=" * 44}')

# MNIST Dataset
train_dataset = datasets.MNIST(root='./mnist_data/',
                               train=True,
                               transform=transforms.ToTensor(),
                               download=True)

test_dataset = datasets.MNIST(root='./mnist_data/',
                              train=False,
                              transform=transforms.ToTensor())

# Data Loader (Input Pipeline)
train_loader = data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size,
                                           shuffle=True)

test_loader = data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size,
                                          shuffle=False)


from torch.nn.modules.utils import _pair

class LocallyConnected2d(nn.Module):
    def __init__(self, in_channels, out_channels, output_size, kernel_size, stride, bias=False):
        super(LocallyConnected2d, self).__init__()
        output_size = _pair(output_size)
        self.weight = nn.Parameter(
            torch.randn(1, out_channels, in_channels, output_size[0], output_size[1], kernel_size**2)
        )
        if bias:
            self.bias = nn.Parameter(
                torch.randn(1, out_channels, output_size[0], output_size[1])
            )
        else:
            self.register_parameter('bias', None)
        self.kernel_size = _pair(kernel_size)
        self.stride = _pair(stride)
        
    def forward(self, x):
        _, c, h, w = x.size()
        kh, kw = self.kernel_size
        dh, dw = self.stride
        x = x.unfold(2, kh, dh).unfold(3, kw, dw)
        x = x.contiguous().view(*x.size()[:-2], -1)
        # Sum in in_channel and kernel_size dims
        out = (x.unsqueeze(1) * self.weight).sum([2, -1])
        if self.bias is not None:
            out += self.bias
        return out


batch_size = 5
in_channels = 3
h, w = 24, 24

output_size = 22
kernel_size = 3
stride = 1


class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        self.conv1=nn.Conv2d(1, 10, kernel_size=5)
        self.conv2=nn.Conv2d(10,20, kernel_size=3, padding=1)
        self.conv3=nn.Conv2d(20,30, kernel_size=3, padding=1)
        self.local=LocallyConnected2d(30, 40, 22, 3, 1, bias=True)
        self.mp=nn.MaxPool2d(2)
        self.fc1=nn.Linear(19360, 1024)
        self.fc2=nn.Linear(1024, 10)

    def forward(self, x):
        in_size=x.size(0)
        x = F.relu(self.mp(self.conv1(x)))
        #print("1",x.size())
        x = F.relu(self.mp(self.conv2(x)))
        #print("2",x.size())
        x = F.relu(self.mp(self.conv3(x)))
        x=self.local(x)
        #print("3", x.size())
        x = x.view(in_size, -1)
        #print("4", x.size())
        x = self.fc1(x)
        x = self.fc2(x)
        return F.log_softmax(x)


model = Net()
model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.5)


def train(epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 10 == 0:
            print('Train Epoch: {} | Batch Status: {}/{} ({:.0f}%) | Loss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))


def test():
    model.eval()
    test_loss = 0
    correct = 0
    for data, target in test_loader:
        data, target = data.to(device), target.to(device)
        output = model(data)
        # sum up batch loss
        test_loss += criterion(output, target).item()
        # get the index of the max
        pred = output.data.max(1, keepdim=True)[1]
        correct += pred.eq(target.data.view_as(pred)).cpu().sum()

    test_loss /= len(test_loader.dataset)
    print(f'===========================\nTest set: Average loss: {test_loss:.4f}, Accuracy: {correct}/{len(test_loader.dataset)} '
          f'({100. * correct / len(test_loader.dataset):.0f}%)')


if __name__ == '__main__':
    since = time.time()
    for epoch in range(1, 10):
        epoch_start = time.time()
        train(epoch)
        m, s = divmod(time.time() - epoch_start, 60)
        print(f'Training time: {m:.0f}m {s:.0f}s')
        test()
        m, s = divmod(time.time() - epoch_start, 60)
        print(f'Testing time: {m:.0f}m {s:.0f}s')

    m, s = divmod(time.time() - since, 60)
    print(f'Total Time: {m:.0f}m {s:.0f}s\nModel was trained on {device}!')



Training MNIST Model on cuda


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:100: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


Train Epoch: 1 | Batch Status: 0/60000 (0%) | Loss: 2.417682
Train Epoch: 1 | Batch Status: 640/60000 (1%) | Loss: 142.515106
Train Epoch: 1 | Batch Status: 1280/60000 (2%) | Loss: 8503.079102
Train Epoch: 1 | Batch Status: 1920/60000 (3%) | Loss: 19265.632812
Train Epoch: 1 | Batch Status: 2560/60000 (4%) | Loss: 2055.636719
Train Epoch: 1 | Batch Status: 3200/60000 (5%) | Loss: 1266.393188
Train Epoch: 1 | Batch Status: 3840/60000 (6%) | Loss: 784.220947
Train Epoch: 1 | Batch Status: 4480/60000 (7%) | Loss: 649.270508
Train Epoch: 1 | Batch Status: 5120/60000 (9%) | Loss: 476.980255
Train Epoch: 1 | Batch Status: 5760/60000 (10%) | Loss: 249.833191
Train Epoch: 1 | Batch Status: 6400/60000 (11%) | Loss: 162.226379
Train Epoch: 1 | Batch Status: 7040/60000 (12%) | Loss: 70.106705
Train Epoch: 1 | Batch Status: 7680/60000 (13%) | Loss: 100.403236
Train Epoch: 1 | Batch Status: 8320/60000 (14%) | Loss: 55.751686
Train Epoch: 1 | Batch Status: 8960/60000 (15%) | Loss: 40.573177
Train Ep